# Assignment 3: Traveling Salesman Problem Using Simulated Annealing

In [ ]:
import csv
from functools import wraps, reduce
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import seaborn as sns
import time

## 0. Functions

0.1 Set up a decorator function timeit to measure and print the execution time of a wrapped function. 

In [ ]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

0.2 Set up a function that reads and parses the data from tsp files, and returns the list of city coordinates.

In [ ]:
def read_tsp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    cities = []
    for line in lines:
        parts = line.split()
        if len(parts) == 3 and parts[0].isdigit():
            cities.append((float(parts[1]), float(parts[2])))

    return cities

0.3 Set up a function which normalises the list of city coordinates to 0-1 coordinates. 

In [ ]:
def normalise_dist(cities):
    # lambda function to split the coordinates into separate lists, 'x_coord','y_coord' 
    devider = lambda cities: ([city[0] for city in cities], [city[1] for city in cities])
    x_coord, y_coord = devider(cities)
    # get minmax values of both coordinates
    minmax_x = (min(x_coord), max(x_coord))
    minmax_y = (min(y_coord), max(y_coord))
    # lambda functioon to normalize the coordinates with the range 
    normalizer = lambda minmax_x, x_coord: [(x - minmax_x[0]) / (minmax_x[1] - minmax_x[0]) for x in x_coord]
    # get normalized values of both coordinates 
    normalized_x = normalizer(minmax_x, x_coord)
    normalized_y = normalizer(minmax_y, y_coord)
    # lambda function to recouple normalized x and y coordinates for each city
    reconstructor = lambda a, b: (a, b)
    # final normalized list
    normalised = list(map(reconstructor, normalized_x, normalized_y))
    return normalised

0.4 Set up a function which reverses the normalization of the list of city coordinates from 0-1 coordinates back to the original values.

In [ ]:
def denormalise_dist(cities, norm_coord):
    # lambda function to split the coordinates into separate lists, 'x_coord','y_coord' 
    devider = lambda cities: ([city[0] for city in cities], [city[1] for city in cities])
    x_coord, y_coord = devider(cities)
    x_norm , y_norm = devider(norm_coord)
    # get minmax values of both coordinates
    minmax_x = (min(x_coord), max(x_coord))
    minmax_y = (min(y_coord), max(y_coord))
    # lambda function to calculate the original coordinates
    denormalize = lambda minmax, norm_coords: [x * (minmax[1] - minmax[0]) + minmax[0] for x in norm_coords]
    denormalized_x = denormalize(minmax_x, x_norm)
    denormalized_y = denormalize(minmax_y, y_norm)
    # lambda function to recouple normadelized x and y coordinates for each city
    reconstructor = lambda a, b: (a, b)
    denormalised = list(map(reconstructor, denormalized_x, denormalized_y))
    return denormalised

0.5 Set up a function which calculates the total distance covered in a given tour of the cities.

In [ ]:
def calculate_total_distance(tour, cities):
    total_distance = 0
    for i in range(len(tour) - 1):
        city1 = cities[tour[i]]
        city2 = cities[tour[i + 1]]
        distance = math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)
        total_distance += distance

    # return to the starting city
    total_distance += math.sqrt((cities[tour[-1]][0] - cities[tour[0]][0])**2 +
                                (cities[tour[-1]][1] - cities[tour[0]][1])**2)

    return total_distance

0.6. Set up functions to ammend the tours 

0.6.1 Set up a function which "moves" a single city's position within a given tour from i to j  

In [ ]:
def move_city(tour):
    i, j = random.sample(range(1, len(tour)-1), 2)
    city = tour.pop(i)
    tour.insert(j, city)
    return tour

0.6.2 Set up a function which "moves" a whole section of the tour to a new (random) position 

In [ ]:
def move_part(route):
    i, j = sorted(random.sample(range(1, len(route)-1), 2))
    route_slice = route[i:j]
    del route[i:j]
    if len(route) - 1 > 0:
        new_position = random.randint(1, len(route) - 1)
    else:
        new_position = 1
    route[new_position:new_position] = route_slice
    return route

0.6.3 Set up a function to implement a 2-opt swap operation for a deterministic algorithm. The 2-opt swap is a local search heuristic that attempts to improve a given route by reversing a portion of it. 

In [ ]:
def two_opt(tour):
    # apply 2-opt move
    i, j = random.sample(range(len(tour)), 2)
    i, j = sorted([i, j])
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

In [ ]:
def two_opt_swap(route, i, j):
    new_route = route[:i] + route[i:j + 1][::-1] + route[j + 1:]
    return new_route

In [ ]:
def optimise_with_2opt(route, cities):
    """The determenistic alg for 2-opt swap on best result."""
    improvement = True
    best_route = route
    best_distance = calculate_total_distance(route, cities)
    while improvement:
        improvement = False
        for i in range(1, len(best_route) - 1):
            for j in range(i + 1, len(best_route)):
                new_route = two_opt_swap(best_route, i, j)
                new_distance = calculate_total_distance(new_route, cities)
                if new_distance < best_distance:
                    best_distance = new_distance
                    best_route = new_route
                    improvement = True
                    break
            if improvement:
                break

    return best_route

0.7 Set up the cooling schedule functions

0.7.1 Logarithmic cooling

In [ ]:
def Hoffmann_Salamon_cooling(iter, D = 10):
        return (D-1)/math.log(iter + 2)

0.7.2 Exponential cooling

In [ ]:
def exponential_cooling(temp, rate):
        return temp*rate

0.7.3 Fast cooling

In [ ]:
def fast_cooling(iter, initial):
        return initial/(1+iter)

0.9. Set up functions to run versions of the simulated annealing method

0.9.1 Intitial - single run test settup - with the four corrective strategies: 2-opt, move city, move part, reverse two adjacent nodes 

In [ ]:
def simulated_annealing_initial(cities):
    """Simulated annealing algorithm"""
    num_cities = len(cities)
    current_tour = list(range(num_cities))
    current_distance = calculate_total_distance(current_tour, cities)
    best_tour = current_tour[:]
    best_distance = current_distance
    temp_array = []
    prob_array = []
    t = []
    fitnes = []
    fitnes_t = []
    temp_t = []
    bb = []
    temp_list = []
    p0 = 0.95
    max_length = 250
    no_progress = 0
    a = [1,1,2,3,3,4]
    
    """Outer loop to decrease schedule"""
    for inner_ier in range(max_length):
        new_tour = current_tour[:]
        u = random.sample(a, 1)
        if u[0] == 1:
            # 2opt swap
            tik = 1
            i, j = sorted(random.sample(range(1, num_cities), 2))
            new_tour[i:j+1] = list(reversed(new_tour[i:j+1]))
        elif u[0] == 2:
            #move city 
            tik = 2
            new_tour = move_city(new_tour)
        elif u[0] == 3:
            #move part
            tik = 3
            new_tour = move_part(new_tour)
        elif u[0] == 4:
            # Reverse two adjacent nodes
            tik = 4
            i, j = random.sample(range(1, num_cities), 2)
            tem = new_tour[i]
            try: 
                new_tour[i] = new_tour[i+1]
                new_tour[i+1] = tem
            except: 
                new_tour[i] = new_tour[i-1]
                new_tour[i-1] = tem
        # calculate the new tour distance
        new_distance = calculate_total_distance(new_tour, cities)
        delta_distance = new_distance - current_distance
        # decide whether to accept the new tour
        if delta_distance <= 0:
            fitnes.append(new_distance)
            bb.append(best_distance)
            current_tour = new_tour
            current_distance = new_distance
            # update the best tour if needed
            if new_distance < best_distance:
                best_tour = new_tour
                best_distance = new_distance
                #print(best_distance)
                no_progress =0
            else:
                no_progress += 1
        else:
                temp_list.append((-delta_distance)/np.log(p0))                     
      
    return temp_list

0.9.2 Alternative set up the function to run the simulated annealing method on a given TSP set up, with a defined intitial temperature parameter value and cooling schedule, taking into account 2-opt and move city optimizations ('tomc')

In [ ]:
def simulated_annealing_tomc(cities, initial_temperature=1000, cooling_rate=0.995, num_iterations=10000):
    num_cities = len(cities)
    current_tour = list(range(num_cities))
    random.shuffle(current_tour)
    current_distance = calculate_total_distance(current_tour, cities)

    best_tour = current_tour[:]
    best_distance = current_distance

    temperature = initial_temperature

    for iteration in range(num_iterations):
        # Choose a random elementary edit
        if random.uniform(0, 1) < 0.5:
            new_tour = two_opt(current_tour)
        else:
            new_tour = move_city(current_tour)

        # Calculate the new tour distance
        new_distance = calculate_total_distance(new_tour, cities)

        # Decide whether to accept the new tour
        if new_distance < current_distance or random.uniform(0, 1) < math.exp((current_distance - new_distance) / temperature):
            current_tour = new_tour
            current_distance = new_distance

            # Update the best tour if needed
            if new_distance < best_distance:
                best_tour = new_tour
                best_distance = new_distance

        # Cool down the temperature
        temperature *= cooling_rate

    return best_tour, best_distance

0.9.3 Set up the function to run the default simulated annealing method on a given TSP set up, with a defined intitial temperature parameter value and exponential cooling schedule

In [ ]:
@timeit
def simulated_annealing(cities, initial_temperature, cooling_rate):

    num_cities = len(cities)
    current_tour = list(range(num_cities))
    current_distance = calculate_total_distance(current_tour, cities)
    best_tour = current_tour[:]
    best_distance = current_distance
    temperature = initial_temperature
    temp_array = []
    prob_array = []
    t = []
    fitnes = []
    fitnes_t = []
    temp_t = []
    bb = []
    no_progress = 0
    
    """Outer loop for the cooling schedule"""
    for inner_ier in range(3000):
        
        a = [1,1,2,3,3,4]
        max_iterations = 3000

        """Inner loop (Markov Chain length)"""
        for iteration in range(max_iterations):
        
            new_tour = current_tour[:]
            u = random.sample(a, 1)
        
            if u[0] == 1:
            # 2opt swap
                tik = 1
                i, j = sorted(random.sample(range(1, num_cities), 2))
                new_tour[i:j+1] = list(reversed(new_tour[i:j+1]))
            elif u[0] == 2:
            # move city 
                tik = 2
                new_tour = move_city(new_tour)
            elif u[0] == 3:
            # move part
                tik = 3
                new_tour = move_part(new_tour)
            elif u[0] == 4:
                # reverse two adjacent neigbours
                tik = 4
                i, j = random.sample(range(1, num_cities), 2)
                tem = new_tour[i]
                try: 
                    new_tour[i] = new_tour[i+1]
                    new_tour[i+1] = tem
                except: 
                    new_tour[i] = new_tour[i-1]
                    new_tour[i-1] = tem
                
            # calculate the new tour distance
            new_distance = calculate_total_distance(new_tour, cities)
            delta_distance = new_distance - current_distance
        
            # decide whether to accept the new tour
            if delta_distance <= 0:
                fitnes.append(new_distance)
                fitnes_t.append((inner_ier)*max_iterations +iteration) 
                bb.append(best_distance)
                current_tour = new_tour
                current_distance = new_distance
                # update the best tour if needed
                if new_distance < best_distance:
                    best_tour = new_tour
                    best_distance = new_distance
                    print(best_distance)
                    no_progress =0
                else:
                    no_progress += 1
            else:
                no_progress += 1
                argument = -delta_distance / temperature
                random_u = random.uniform(0, 1)
                prob_array.append(math.exp(argument))
                t.append((inner_ier)*max_iterations +iteration) 
                if math.exp(argument) > random_u:
                    current_tour = new_tour
                    current_distance = new_distance

        temperature = exponential_cooling(temperature, cooling_rate)
        
        # reannealing
        if temperature < 0.1 and no_progress > 50:
            best_route_2opt = optimise_with_2opt(best_tour, cities)
            best_dist_2opt = calculate_total_distance(best_route_2opt, cities)
            if best_dist_2opt == best_distance:
                temperature = 3
                print("reannealing")
            
        temp_array.append(temperature)
        temp_t.append(inner_ier)                       
      
    return best_tour, best_distance ,temp_array, prob_array, t, fitnes, fitnes_t, temp_t, bb

0.9.4. Simualated annealing with the list-based cooling schedule 

In [ ]:
@timeit
def simulated_annealing_list(cities, initial_temperature, cooling_rate, colling_schedule_func, temper_list):
    """Simulated annealing algorithm"""
    num_cities = len(cities)
    current_tour = list(range(num_cities))
    current_distance = calculate_total_distance(current_tour, cities)
    best_tour = current_tour[:]
    best_distance = current_distance
    list_temper = temper_list.copy()
    temperature = np.max(list_temper)
    temp_array = []
    prob_array = []
    t = []
    fitnes = []
    fitnes_t = []
    temp_t = []
    bb = []
    no_progress = 0
    """Outer loop to decrease schedule"""
    for inner_ier in range(2000):
      """Inner loop (Markov Chain length)"""
      temp_list = []
      a = [1,1,2,3,3,4]
      inner_t = 0
      c = 0
      for iteration in range(1000):
        new_tour = current_tour[:]
        u = random.sample(a, 1)
        if u[0] == 1:
            # 2opt swap
            tik = 1
            i, j = sorted(random.sample(range(1, num_cities), 2))
            new_tour[i:j+1] = list(reversed(new_tour[i:j+1]))
        elif u[0] == 2:
            #move city 
            tik = 2
            new_tour = move_city(new_tour)
        elif u[0] == 3:
            #move part
            tik = 3
            new_tour = move_part(new_tour)
        elif u[0] == 4:
            # Reverse two adjacent neigbours
            tik = 4
            i, j = random.sample(range(1, num_cities), 2)
            tem = new_tour[i]
            try: 
                new_tour[i] = new_tour[i+1]
                new_tour[i+1] = tem
            except: 
                new_tour[i] = new_tour[i-1]
                new_tour[i-1] = tem
        # calculate the new tour distance
        new_distance = calculate_total_distance(new_tour, cities)
        delta_distance = new_distance - current_distance
        # decide whether to accept the new tour
        if delta_distance <= 0:
            fitnes.append(new_distance)
            fitnes_t.append((inner_ier)*2000 +iteration) 
            current_tour = new_tour
            current_distance = new_distance
            # update the best tour if needed
            if new_distance < best_distance:
                best_tour = new_tour
                best_distance = new_distance
                #print(best_distance)
                no_progress =0
            else:
                no_progress += 1
        else:
            no_progress += 1
            argument = -delta_distance / temperature
            random_u = random.uniform(0, 1)
            prob_array.append(math.exp(argument))
            t.append((inner_ier)*2000 +iteration) 
            if math.exp(argument) > random_u:
                #print((inner_t-delta_distance)/np.log(random_u))
                inner_t += (-delta_distance)/np.log(random_u)
                c +=1
                #temp_list.append((inner_t-delta_distance)/np.log(random_u))
                #temp_list.append(inner_t)
                #print(inner_t)
                current_tour = new_tour
                current_distance = new_distance
            
      #temperature = colling_schedule_func(inner_ier, initial_temperature, temperature, cooling_rate)
      if c > 0:
            list_temper.remove(max(list_temper))
            list_temper.append(inner_t/c)
            temperature = np.max(list_temper)
            #print(temperature)
      if inner_ier % 100 == 0:
                temp_array.append(temperature)
                bb.append(best_distance)
                #print(temperature)
                temp_t.append(inner_ier)      
      #temperature = exponential_cooling(temperature, cooling_rate)            
      
    return best_tour, best_distance ,temp_array, prob_array, t, fitnes, fitnes_t, temp_t, bb

0.9.5. Simulated annealing with a cool_func implementation 

In [ ]:
@timeit
def simulated_annealing_cooling_check(cities, initial_temperature, cooling_rate, cool_func):
    """Simulated annealing algorithm"""
    num_cities = len(cities)
    current_tour = list(range(num_cities))
    current_distance = calculate_total_distance(current_tour, cities)
    best_tour = current_tour[:]
    best_distance = current_distance
    temperature = initial_temperature
    temp_array = []
    prob_array = []
    t = []
    fitnes = []
    fitnes_t = []
    temp_t = []
    bb = []
    no_progress = 0
    """Outer loop to decrease schedule"""
    for inner_ier in range(2000):
      a = [1,1,2,3,3,4]
      """Inner loop (Markov Chain length)"""
      for iteration in range(1000):
        new_tour = current_tour[:]
        u = random.sample(a, 1)
        if u[0] == 1:
            # 2opt swap
            tik = 1
            i, j = sorted(random.sample(range(1, num_cities), 2))
            new_tour[i:j+1] = list(reversed(new_tour[i:j+1]))
        elif u[0] == 2:
            #move city 
            tik = 2
            new_tour = move_city(new_tour)
        elif u[0] == 3:
            #move part
            tik = 3
            new_tour = move_part(new_tour)
        elif u[0] == 4:
            # Reverse two adjacent neigbours
            tik = 4
            i, j = random.sample(range(1, num_cities), 2)
            tem = new_tour[i]
            try: 
                new_tour[i] = new_tour[i+1]
                new_tour[i+1] = tem
            except: 
                new_tour[i] = new_tour[i-1]
                new_tour[i-1] = tem
        # calculate the new tour distance
        new_distance = calculate_total_distance(new_tour, cities)
        delta_distance = new_distance - current_distance
        # decide whether to accept the new tour
        if delta_distance <= 0:
            #a.append(tik)
            fitnes.append(new_distance)
            fitnes_t.append((inner_ier)*2000 +iteration) 
            current_tour = new_tour
            current_distance = new_distance
            # update the best tour if needed
            if new_distance < best_distance:
                best_tour = new_tour
                best_distance = new_distance
                #print(best_distance)
                no_progress = 0
            else:
                no_progress += 1
        else:
            no_progress += 1
            argument = -delta_distance / temperature
            random_u = random.uniform(0, 1)
            prob_array.append(math.exp(argument))
            t.append((inner_ier)*2000 +iteration) 
            if math.exp(argument) > random_u:
                current_tour = new_tour
                current_distance = new_distance
            
      temperature = cool_func(inner_ier, initial_temperature, temperature, cooling_rate)
      # reannealing
      if temperature < 0.7 and no_progress > 6000:
        temperature = 3
      if inner_ier % 100 == 0:
                temp_array.append(temperature)
                bb.append(best_distance)
                #print(temperature)
                temp_t.append(inner_ier)  
    return best_tour, best_distance ,temp_array, prob_array, t, fitnes, fitnes_t, temp_t, bb

0.9.6 Set up the function to run the default simulated annealing method on a given TSP set up, with a defined intitial temperature parameter value and cooling schedule and a changing length of Markov chains 

In [ ]:
@timeit
def simulated_annealing_mar_len(cities, initial_temperature, cooling_rate):

    num_cities = len(cities)
    mc_iterations = 10
    best_tours_mc_l = list(range(mc_iterations))
    best_distances_mc_l = []
    temp_array_list = list(range(mc_iterations))
    temp_t_list = list(range(mc_iterations))
    prob_array_list = list(range(mc_iterations))
    t_list = list(range(mc_iterations))
    fitness_list = list(range(mc_iterations))
    fitness_t_list = list(range(mc_iterations))
    best_list = list(range(mc_iterations))

    """Lenght of Markov chain spec."""
    for l in range(mc_iterations):
        max_iterations = 500 + l * 500
        print(max_iterations)
        
        current_tour = list(range(num_cities))
        temp_array = []
        prob_array = []
        t = []
        fitnes = []
        fitnes_t = []
        temp_t = []
        bb = []
        no_progress = 0
        best_tour = current_tour[:]
        current_distance = calculate_total_distance(current_tour, cities)
        best_distance = current_distance
        temperature = initial_temperature
        
        """Outer loop for the cooling schedule"""
        for inner_ier in range(3000):

            a = [1,1,2,3,3,4]

            """Inner loop (Markov Chain length)"""
            for iteration in range(max_iterations):

                new_tour = current_tour[:]
                u = random.sample(a, 1)

                if u[0] == 1:
                # 2opt swap
                    tik = 1
                    i, j = sorted(random.sample(range(1, num_cities), 2))
                    new_tour[i:j+1] = list(reversed(new_tour[i:j+1]))
                elif u[0] == 2:
                # move city 
                    tik = 2
                    new_tour = move_city(new_tour)
                elif u[0] == 3:
                # move part
                    tik = 3
                    new_tour = move_part(new_tour)
                elif u[0] == 4:
                    # reverse two adjacent neigbours
                    tik = 4
                    i, j = random.sample(range(1, num_cities), 2)
                    tem = new_tour[i]
                    try: 
                        new_tour[i] = new_tour[i+1]
                        new_tour[i+1] = tem
                    except: 
                        new_tour[i] = new_tour[i-1]
                        new_tour[i-1] = tem

                # calculate the new tour distance
                new_distance = calculate_total_distance(new_tour, cities)
                delta_distance = new_distance - current_distance

                # decide whether to accept the new tour
                if delta_distance <= 0:
                    fitnes.append(new_distance)
                    fitnes_t.append((inner_ier)*max_iterations +iteration) 
                    bb.append(best_distance)
                    current_tour = new_tour
                    current_distance = new_distance
                    # update the best tour if needed
                    if new_distance < best_distance:
                        best_tour = new_tour
                        best_distance = new_distance
                        # print(best_distance)
                        no_progress =0
                    else:
                        no_progress += 1
                else:
                    no_progress += 1
                    argument = -delta_distance / temperature
                    random_u = random.uniform(0, 1)
                    prob_array.append(math.exp(argument))
                    t.append((inner_ier)*max_iterations +iteration) 
                    if math.exp(argument) > random_u:
                        current_tour = new_tour
                        current_distance = new_distance
                        
            temperature = exponential_cooling(temperature, cooling_rate)

            # reaneiling
            if temperature < 0.1 and no_progress > 50:
                best_route_2opt = optimise_with_2opt(best_tour, cities)
                best_dist_2opt = calculate_total_distance(best_route_2opt, cities)
                if best_dist_2opt == best_distance:
                    temperature = 3
                    print("reannealing")

            temp_array.append(temperature)
            temp_t.append(inner_ier) 
            
        print("Final results for MC length ", max_iterations)
        print(best_tour)
        best_tours_mc_l[l]= best_tour
        print(best_distance)
        best_distances_mc_l.append(best_distance)
        #print(temp_array)
        temp_array_list[l] = temp_array
        #print(temp_t)
        temp_t_list[l]= temp_t
        prob_array_list[l] = prob_array
        t_list[l] = t
        fitness_list[l] = fitnes
        fitness_t_list[l] = fitnes_t
        best_list[l] = bb
            
    return best_tours_mc_l,best_distances_mc_l,temp_array_list,temp_t_list, t_list, fitness_list, fitness_t_list, best_list

0.10 Set up a function to plot a given tour of the cities

In [ ]:
def plot_tour(tour, cities):
    x = [cities[city][0] for city in tour]
    y = [cities[city][1] for city in tour]
    x.append(x[0])
    y.append(y[0])
    plt.figure(figsize=(8, 6))
    plt.plot(x, y, 'o-', label='Tour Path')
    plt.plot([cities[i][0] for i in range(len(cities))], 
             [cities[i][1] for i in range(len(cities))], 
             'o', label='Cities')
    
    plt.title('Traveling Salesman Problem - Simulated Annealing')
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.legend()
    plt.show()

0.11.1 Set up a function to run multiple simulated_annealing simulations

In [ ]:
def run_multiple_simulations(cities, num_simulations=100):
    best_overall_tour = None
    best_overall_distance = float('inf')

    for i in range(num_simulations):
        #print(f"Running Simulation {i + 1}")
        current_tour, current_distance = simulated_annealing(cities)
        if current_distance < best_overall_distance:
            best_overall_tour = current_tour
            best_overall_distance = current_distance

    return best_overall_tour, best_overall_distance

0.11.1 Set up a function to run multiple tomc simulated_annealing simulations

In [ ]:
def run_multiple_simulations_tomc(cities, num_simulations=100):
    best_overall_tour = None
    best_overall_distance = float('inf')

    for i in range(num_simulations):
        #print(f"Running Simulation {i + 1}")
        current_tour, current_distance = simulated_annealing_tomc(cities)
        if current_distance < best_overall_distance:
            best_overall_tour = current_tour
            best_overall_distance = current_distance

    return best_overall_tour, best_overall_distance

0.12 Set up a function to find the best hyperparameters for a given TSP

In [ ]:
# too long to run, delete if not used
def param_search(cities, initial_temp, cooling_rate, in_temp=0.05, col_rate=0.95, max_no_improve=30):
    """ Simulated aneiling to find best hyperparameters for TSP"""
    curr_temp = initial_temp
    curr_rate = cooling_rate
    best_tour, best_distance_init, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing(cities, initial_temp, cooling_rate)
    
    temperature = in_temp
    best_abs = np.inf
    best_initial_temp = 0 
    best_cooling_rate = 0
    no_improve_counter = 0

    log = []

    for k in range(150):
        step_size_temp = 2.5 / (1 + k)  
        step_size_rate = 0.01 / (1 + k)  

        mean_len = []
        improved = False

        for i in range(4):
            u = random.uniform(-step_size_temp, step_size_temp) 
            new_temp = max(curr_temp + u, 0.1) 
            u = random.uniform(-step_size_rate, step_size_rate) 
            new_rate = min(max(curr_rate + u, 0.01), 0.999) 

            for b in range(7):
                best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing(cities, new_temp, new_rate)
                mean_len.append(best_distance)

            mean_len_v = np.mean(mean_len)
            delta_distance = mean_len_v - best_distance_init
            argument = -delta_distance / temperature
            random_u = random.uniform(0, 1)

            if delta_distance < 0 or math.exp(argument) > random_u:
                curr_temp = new_temp
                curr_rate = new_rate
                best_distance_init = mean_len_v
                print(step_size_temp)
                if best_distance_init < best_abs:
                    best_abs = best_distance_init
                    print(f"Abs best:{best_abs}")
                    print(f"cur temp:{temperature}")
                    print(f"new_temp :{new_temp}")
                    print(f"new_rate:{new_rate}")
                    best_initial_temp = curr_temp 
                    best_cooling_rate = curr_rate
                    improved = True

        log.append((k, curr_temp, curr_rate, best_abs))

        if not improved:
            no_improve_counter += 1
        else:
            no_improve_counter = 0

        if no_improve_counter >= max_no_improve:
            print("Stopping early due to no improvement.")
            break

        temperature *= col_rate

    return best_abs, best_initial_temp, best_cooling_rate, log


## 1. Test Runs: General

1.1 Test effect of delta and initial temperature on cooling schedule. 

In [ ]:
temperature = 20
initial_temperature = 20
ar = []
ir = []
pr = []
for i in range(1, 2000):
    temperature =  initial_temperature / (np.log(i + 1))
    ar.append(temperature)
    ir.append(i)
    argument = -0.05 / temperature
    pr.append(math.exp(argument))
plt.plot(ir, ar)
plt.show()
plt.plot(ir, pr)

In [ ]:
temperature = 20
initial_temperature = 40
ar = []
ir = []
pr = []
for i in range(1, 2000):
    temperature =  initial_temperature / (1 + i)
    ar.append(temperature)
    ir.append(i)
    argument = -0.05 / temperature
    pr.append(math.exp(argument))

plt.plot(ir, ar)
plt.show()
plt.plot(ir, pr)

## 2. Test Runs: TSP - Configuration: EIL51

In [ ]:
os.getcwd()

2.1 Set up the file extraction

In [ ]:
# the path to the folder containing the csv file
folder_path = 'TSP-Configurations/'

In [ ]:
# the name of the csv file you want to read
file_name = 'eil51.tsp.txt'

# combine the folder path and file name to create the file path
file_path = folder_path + file_name

2.2 Run the default simulated annealing simulation

2.2.1 Initialization with a fixed cooling rate

In [ ]:
tsp_file_path = file_path
cities = read_tsp_file(tsp_file_path)
random.shuffle(cities)
cities_norm = normalise_dist(cities)
    
initial_temp = 1400
cooling_rate = 0.995 

2.2.2 Estimate the best tour route and distance 

In [ ]:
best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing(cities,initial_temp,cooling_rate)

In [ ]:
best

In [ ]:
min(best)

2.2.3 Decouple the coordinates

In [ ]:
x = [cities[city][0] for city in best_tour]
y = [cities[city][1] for city in best_tour]
x.append(x[0])
y.append(y[0])

2.2.4 Calculate best total distance

In [ ]:
best_distance = calculate_total_distance(best_tour, cities)

2.2.5 Plot the best route

In [ ]:
print("Best Tour:", best_tour)
print("Best Distance:", best_distance)
plot_tour(best_tour, cities)

2.2.6 Apply further optimization options on the best_tour

In [ ]:
best_route_2opt = optimise_with_2opt(best_tour, cities)
best_dist_2opt = calculate_total_distance(best_route_2opt, cities)
print("Best Distance_2opt:", best_dist_2opt)

In [ ]:
coord_denorm = list(zip(x, y))
plot_tour(best_tour, cities_norm)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(temp_t, temp_arr)
plt.title('Traveling Salesman Problem - Simulated Annealing - EL51')
plt.xlabel('Temp_t')
plt.ylabel('Temp_rr')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(t, prob, s=5, label='Data Points')
plt.title('Traveling Salesman Problem - Simulated Annealing - EL51')
plt.xlabel('t')
plt.ylabel('prob')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ft, fit)
plt.title('Traveling Salesman Problem - Simulated Annealing - EL51')
plt.xlabel('ft')
plt.ylabel('fit')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ft, best)
plt.title('Traveling Salesman Problem - Simulated Annealing - EL51')
plt.xlabel('ft')
plt.ylabel('best')
plt.show()

In [ ]:
print(best_dist_2opt)

2.3.X Run the simulated annealing method with the 2-opt and move-city corrections

In [ ]:
best_tour, best_distance = simulated_annealing_tomc(cities)
print("Best Tour:", best_tour)
print("Best Distance:", best_distance)
plot_tour(best_tour, cities)

This is not the result we want.

2.3.2 Run multiple simulations of simulated annealing method with the 2-opt and move-city corrections

In [ ]:
best_tour, best_distance = run_multiple_simulations_tomc(cities, num_simulations=3000)
print("Best Tour:", best_tour)
print("Best Distance:", best_distance)
plot_tour(best_tour, cities)

This is not the result we want.

## 3. Test Runs: TSP - Configuration: A280

In [ ]:
# the name of the csv file you want to read
file_name = 'a280.tsp.txt'

# combine the folder path and file name to create the file path
file_path_a280 = folder_path + file_name

In [ ]:
tsp_file_path_a280 = file_path_a280
cities_a280 = read_tsp_file(tsp_file_path_a280)
random.shuffle(cities_a280)
cities_norm_a280 = normalise_dist(cities_a280)
    
initial_temp = 1400
cooling_rate = 0.995 

3.1 Run simulated annealing method with a fixed cooling rate.

In [ ]:
best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing(cities_a280, initial_temp, cooling_rate)

In [ ]:
x = [cities_a280[city][0] for city in best_tour]
y = [cities_a280[city][1] for city in best_tour]
x.append(x[0])
y.append(y[0])

In [ ]:
best_distance = calculate_total_distance(best_tour, cities_a280)

In [ ]:
print("Best Tour for A280:", best_tour)
print("Best Distance for A280:", best_distance)
plot_tour(best_tour, cities_a280)

In [ ]:
coord_denorm = list(zip(x, y))
plot_tour(best_tour, cities_norm_a280)

In [ ]:
best_route_2opt = optimise_with_2opt(best_tour, cities_a280)
best_dist_2opt = calculate_total_distance(best_route_2opt, cities_a280)
print("Best Distance_2opt:", best_dist_2opt)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(temp_t, temp_arr)
plt.title('Traveling Salesman Problem - Simulated Annealing - A280')
plt.xlabel('Temp_t')
plt.ylabel('Temp_rr')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(t, prob, s=5, label='Data Points')
plt.title('Traveling Salesman Problem - Simulated Annealing - A280')
plt.xlabel('t')
plt.ylabel('prob')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ft, fit)
plt.title('Traveling Salesman Problem - Simulated Annealing - A280')
plt.xlabel('ft')
plt.ylabel('fit')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ft, best)
plt.title('Traveling Salesman Problem - Simulated Annealing - A280')
plt.xlabel('ft')
plt.ylabel('best')
plt.show()

In [ ]:
print(best_dist_2opt)

3.1 Run simulated annealing method with an exponential cooling schedule.

In [ ]:
initial_temp_list = simulated_annealing_initial(cities_a280)

3.2 Run simulated annealing with the list-based cooling schedule

In [ ]:
initial_temp = 0
cooling_rate = 0
best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing_list(cities_a280,initial_temp,cooling_rate, exponential_cooling, initial_temp_list)
best_distance = calculate_total_distance(best_tour, cities_a280)

print("Best Tour for A280 with the exponential cooling schedule:", best_tour)
print("Best Distance for A280 with the exponential cooling schedule:", best_distance)
plot_tour(best_tour, cities_a280)

plt.plot(temp_t, temp_arr)
plt.show()
plt.scatter(t, prob)
plt.show()
plt.plot(ft, fit)
plt.show()
plt.plot(ft, best)
plt.yscale('log')
plt.show()

DESCRIPTION_PLACEHOLDER

In [ ]:
types = [exponential_cooling, fast_cooling]
best_distance_list = []
temp_arrs_list = []
fit_arr_list = []
temp_t_arr_list = []
prob_arr_list = []
t_arr_list = []
ft_arr_list = []
best_arr_list = []
best_tour_arr_list = []
temp_arr_list = []

best_distance_file = 'best_distance_list.csv'
temp_file = 'temp_list.csv'
best_arr_lis = 'best_arr_list.csv'

for _ in range(30):
        tsp_file_path = file_path_a280
        cities = read_tsp_file(tsp_file_path)
        random.shuffle(cities_a280)
        initial_temp_list = simulated_annealing_initial(cities_a280)
        initial_temp = 1400
        cooling_rate = 0.995
        best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing_list(cities_a280,initial_temp,cooling_rate,fast_cooling, initial_temp_list)
        with open(best_distance_file, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([best_distance])
        with open(temp_file, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([temp_arr])
        with open(best_arr_lis, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([best])

In [ ]:
best_distance_file_fast_cooling = 'best_distance_arr_fast_cooling.csv'
temp_file_fast_cooling = 'temp_fast_cooling.csv'
best_arr_fast_cooling = 'best_arr_fast_cooling.csv'

for iteration in range(30):
        tsp_file_path = file_path_a280
        cities = read_tsp_file(tsp_file_path)
        random.shuffle(cities_a280)
        initial_temp = 1400
        cooling_rate = 0.995
        best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best = simulated_annealing_cooling_check(cities_a280, initial_temp, cooling_rate, fast_cooling)

        with open(best_distance_file_fast_cooling, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([best_distance])
        with open(temp_file_fast_cooling, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([temp_arr])
        with open(best_arr_fast_cooling, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([best])

In [ ]:
best_distanceexp= 'best_distance_arr_fexpon.csv'
temp_fileexp= 'temp_fastexpon.csv'
best_arrexp = 'best_arr_fast_expon.csv'
for _ in range(30):
    tsp_file_path = file_path_a280
    cities = read_tsp_file(tsp_file_path)
    random.shuffle(cities_a280)
    initial_temp = 1400
    cooling_rate = 0.995
    best_tour, best_distance, temp_arr, prob, t, fit, ft, temp_t, best  = simulated_annealing_cooling_check(cities_a280,initial_temp,cooling_rate,exponential_cooling)
    
    with open(best_distanceexp, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([best_distance])
    with open(temp_fileexp, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([temp_arr])
    with open(best_arrexp, 'a', newline='') as bd_file:
            bd_writer = csv.writer(bd_file)
            bd_writer.writerow([best])

### 2.3.1 Length of the Markov Chain

Run the simulated annealing method with exponential cooling schedule for varying values of Markov chain length

In [ ]:
best_tours_mc_l,best_distances_mc_l,temp_array_list,temp_t_list, t_list, fitness_list, fitness_t_list, best_list = simulated_annealing_mar_len(cities_a280,initial_temp,cooling_rate)

In [ ]:
# create a new list with counters for each sublist
time_stamp_list = [[i+1 for i in range(len(sublist))] for sublist in best_list]

In [ ]:
# list of labels for legend
labels = ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000']  

plt.figure(figsize=(10, 6))
plt.title('Convergence to Optimal Value Estimate for A280 for Varying Markov Chain Length')

# generate a palette of colors
colors = sns.color_palette("husl", len(temp_t_list))

# plot each pair of arrays with a different color from the palette
for x_values, y_values, color, label in zip(time_stamp_list, best_list, colors, labels):
    plt.plot(x_values, y_values, linestyle='-', linewidth=1, color=color, label=label)

plt.xlabel('Time Steps')  
plt.ylabel('Optimal Value Estimate for the TSP Route Length') 
plt.legend()
plt.show()

In [ ]:
# list of labels for legend
labels = ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000']  

plt.figure(figsize=(10, 6))
plt.title('Convergence to Optimal Value Estimate for A280 for Varying Markov Chain Length (loglog)')

# generate a palette of colors
colors = sns.color_palette("husl", len(temp_t_list))

# plot each pair of arrays with a different color from the palette
for x_values, y_values, color, label in zip(time_stamp_list, best_list, colors, labels):
    plt.loglog(x_values, y_values, linestyle='-', linewidth=1, color=color, label=label)

plt.xlabel('Time Steps')  
plt.ylabel('Optimal Value Estimate for the TSP Route Length') 
plt.legend()
plt.show()

## 3. Test Runs: TSP - Configuration: PCB442

In [ ]:
# the name of the csv file you want to read
file_name = 'pcb442.tsp.txt'

# combine the folder path and file name to create the file path
file_path_pcb442 = folder_path + file_name

In [ ]:
tsp_file_path_pcb442 = file_path_pcb442
cities_pcb442 = read_tsp_file(tsp_file_path_pcb442)
random.shuffle(cities_pcb442)
cities_norm_pcb442 = normalise_dist(cities_pcb442)
    
initial_temp = 1400
cooling_rate = 0.995 

In [ ]:
best_tour_pcb442, best_distance_pcb442, temp_arr, prob, t, fit, ft, temp_t, best_pcb442 = simulated_annealing(cities_pcb442,initial_temp,cooling_rate)

In [ ]:
x = [cities_pcb442[city][0] for city in best_tour_pcb442]
y = [cities_pcb442[city][1] for city in best_tour_pcb442]
x.append(x[0])
y.append(y[0])

In [ ]:
best_distance_pcb442 = calculate_total_distance(best_tour_pcb442, cities_pcb442)

In [ ]:
print("Best Tour for PCB442:", best_tour_pcb442)
print("Best Distance for PCB442:", best_distance_pcb442)
plot_tour(best_tour_pcb442, cities_pcb442)

In [ ]:
coord_denorm = list(zip(x, y))
plot_tour(best_tour_pcb442, cities_norm_pcb442)

In [ ]:
best_route_2opt_pcb442 = optimise_with_2opt(best_tour_pcb442, cities_pcb442)
best_dist_2opt_pcb442 = calculate_total_distance(best_route_2opt_pcb442, cities_pcb442)
print("Best Distance_2opt for PCB442:", best_dist_2opt_pcb442)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(temp_t, temp_arr)
plt.title('Traveling Salesman Problem - Simulated Annealing - PCB442')
plt.xlabel('Temp_t')
plt.ylabel('Temp_rr')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(t, prob, s=5, label='Data Points')
plt.title('Traveling Salesman Problem - Simulated Annealing - PCB442')
plt.xlabel('t')
plt.ylabel('prob')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ft, fit)
plt.title('Traveling Salesman Problem - Simulated Annealing - PCB442')
plt.xlabel('ft')
plt.ylabel('fit')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ft, best_pcb442)
plt.title('Traveling Salesman Problem - Simulated Annealing - PCB442')
plt.xlabel('ft')
plt.ylabel('best')
plt.show()